In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
%matplotlib inline

In [3]:
folder_path = 'data'

# Check if the folder exists
if os.path.exists(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    
    # Print the list of file names
    for file in files:
        print(file)
else:
    print(f"The folder '{folder_path}' does not exist.")


AAPL_2010-01-01_2023-01-01.csv
AMC_2010-01-01_2023-01-01.csv
AMZN_2010-01-01_2023-01-01.csv
BB_2010-01-01_2023-01-01.csv
GME_2010-01-01_2023-01-01.csv
GOOG_2010-01-01_2023-01-01.csv
MSFT_2010-01-01_2023-01-01.csv
PLTR_2010-01-01_2023-01-01.csv
QQQ_2010-01-01_2023-01-01.csv
RIVN_2010-01-01_2023-01-01.csv
SOFI_2010-01-01_2023-01-01.csv
SPY_2010-01-01_2023-01-01.csv
TSLA_2010-01-01_2023-01-01.csv


In [4]:
# file_path = 'data/AAPL_2010-01-01_2023-01-01.csv'
# stock_data = pd.read_csv(file_path)
# stock_data['Date'] = pd.to_datetime(stock_data['Date'])

In [5]:
# List of stock data file paths and corresponding stock names
stock_files = [
    {'file_path': 'data/AAPL_2010-01-01_2023-01-01.csv', 'stock_name': 'AAPL'},
    {'file_path': 'data/AMC_2010-01-01_2023-01-01.csv', 'stock_name': 'AMC'},
    {'file_path': 'data/AMZN_2010-01-01_2023-01-01.csv', 'stock_name': 'AMZN'},
    {'file_path': 'data/BB_2010-01-01_2023-01-01.csv', 'stock_name': 'BB'},
    {'file_path': 'data/GME_2010-01-01_2023-01-01.csv', 'stock_name': 'GME'},
    {'file_path': 'data/GOOG_2010-01-01_2023-01-01.csv', 'stock_name': 'GOOG'},
    {'file_path': 'data/MSFT_2010-01-01_2023-01-01.csv', 'stock_name': 'MSFT'},
    {'file_path': 'data/PLTR_2010-01-01_2023-01-01.csv', 'stock_name': 'PLTR'},
    {'file_path': 'data/QQQ_2010-01-01_2023-01-01.csv', 'stock_name': 'QQQ'},
    {'file_path': 'data/RIVN_2010-01-01_2023-01-01.csv', 'stock_name': 'RIVN'},
    {'file_path': 'data/SOFI_2010-01-01_2023-01-01.csv', 'stock_name': 'SOFI'},
    {'file_path': 'data/SPY_2010-01-01_2023-01-01.csv', 'stock_name': 'SPY'},
    {'file_path': 'data/TSLA_2010-01-01_2023-01-01.csv', 'stock_name': 'TSLA'}
]

# Load the selected stock data
selected_stock_data = None

# UI Components
stock_dropdown = widgets.Dropdown(
    options=[(stock['stock_name'], i) for i, stock in enumerate(stock_files)],
    description='Select Stock:',
    disabled=False,
)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False
)

# Set default values for price and volume range sliders
default_min_close = 0  # Set a default minimum close price
default_max_close = 100  # Set a default maximum close price
default_min_volume = 0  # Set a default minimum volume
default_max_volume = 1000000  # Set a default maximum volume

price_range_slider = widgets.FloatRangeSlider(
    value=[default_min_close, default_max_close],
    min=default_min_close,
    max=default_max_close,
    step=0.1,
    description='Price Range:',
    continuous_update=False
)

volume_range_slider = widgets.FloatRangeSlider(
    value=[default_min_volume, default_max_volume],
    min=default_min_volume,
    max=default_max_volume,
    step=10000,
    description='Volume Range:',
    continuous_update=False
)

chart_type_dropdown = widgets.Dropdown(
    options=['Line Chart', 'Candlestick Chart'],
    value='Line Chart',
    description='Chart Type:',
    disabled=False,
)

filter_button = widgets.Button(description='Apply Filters')
output = widgets.Output()

# Inside the load_selected_stock_data function
def load_selected_stock_data(change):
    global selected_stock_data
    stock_index = change.new
    if stock_index is not None:
        file_path = stock_files[stock_index]['file_path']
        selected_stock_data = pd.read_csv(file_path)
        selected_stock_data['Date'] = pd.to_datetime(selected_stock_data['Date'])
        
        # Set the start and end date pickers to the dataset's start and end dates
        if not selected_stock_data.empty:
            min_date = selected_stock_data['Date'].min()
            max_date = selected_stock_data['Date'].max()
            start_date_picker.value = min_date.date()
            end_date_picker.value = max_date.date()
        # clear_output(wait=True)
        # print(f"Loaded {stock_files[stock_index]['stock_name']} stock data.")
        
        # Update price_range_slider and volume_range_slider based on real data range if valid
        if 'Close' in selected_stock_data.columns and 'Volume' in selected_stock_data.columns:
            min_volume = selected_stock_data['Volume'].min()
            max_volume = selected_stock_data['Volume'].max()
            if not pd.isna(min_volume) and not pd.isna(max_volume):
                if min_volume <= volume_range_slider.max:
                    volume_range_slider.min = min_volume
                if max_volume >= volume_range_slider.min:
                    volume_range_slider.max = max_volume
                volume_range_slider.value = [min_volume, max_volume]
            else:
                # Fallback in case of invalid data
                volume_range_slider.min = 0
                volume_range_slider.max = 1
                volume_range_slider.value = [0, 1]


# Observe changes in the selected stock dropdown
stock_dropdown.observe(load_selected_stock_data, names='value')


# Function to Update Output
def filter_data(b):
    with output:
        clear_output(wait=True)
        plt.clf()  # Clear the current figure
        
        if selected_stock_data is None or selected_stock_data.empty:
            print("Please select a stock and load the data.")
            return

        filtered_data = selected_stock_data[
            (selected_stock_data['Date'] >= pd.to_datetime(start_date_picker.value)) & 
            (selected_stock_data['Date'] <= pd.to_datetime(end_date_picker.value)) &
            (selected_stock_data['Close'] >= price_range_slider.value[0]) & 
            (selected_stock_data['Close'] <= price_range_slider.value[1]) &
            (selected_stock_data['Volume'] >= volume_range_slider.value[0]) & 
            (selected_stock_data['Volume'] <= volume_range_slider.value[1])
        ]

        if filtered_data.empty:
            print("No data to display for the selected filters.")
            return
        # print(filtered_data)

        if chart_type_dropdown.value == 'Line Chart':
            # Plot the stock price as a line chart
            plt.figure(figsize=(10, 4))
            plt.plot(filtered_data['Date'], filtered_data['Close'], label='Close Price')
            plt.xlabel('Date')
            plt.ylabel('Price')
            plt.title(f"{stock_files[stock_dropdown.value]['stock_name']} Stock Price")
            plt.legend()
            plt.show()

        elif chart_type_dropdown.value == 'Candlestick Chart':
            fig = go.Figure(data=[go.Candlestick(
                x=filtered_data['Date'],
                open=filtered_data['Open'],
                high=filtered_data['High'],
                low=filtered_data['Low'],
                close=filtered_data['Close']
            )])

            fig.update_layout(
                title=f"{stock_files[stock_dropdown.value]['stock_name']} Candlestick Chart",
                title_font=dict(color='white'),
                title_x=0.5,  
                title_y=0.95, 
                xaxis_title='Date',
                yaxis_title='Price',
                xaxis_rangeslider_visible=False,
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(245, 245, 245, 1)',
                margin=dict(l=40, r=40, t=40, b=40),
                hovermode='closest',
                xaxis=dict(
                    showline=True,
                    showgrid=False,
                    showticklabels=True,
                    tickfont=dict(color='white'),
                    title_font=dict(color='white')
                ),
                yaxis=dict(
                    showline=True,
                    showgrid=True,
                    showticklabels=True,
                    gridcolor='gray',
                    tickfont=dict(color='white'),
                    title_font=dict(color='white')
                ),
                showlegend=True,
                autosize=True,
                legend_title_font=dict(color='white'),  # Set legend title color to white
                legend_font=dict(color='white')
            )

            fig.show()

# Link the filter button to the filter_data function
filter_button.on_click(filter_data)

# Display UI components
display(stock_dropdown, start_date_picker, end_date_picker, price_range_slider, volume_range_slider, chart_type_dropdown, filter_button, output)


Dropdown(description='Select Stock:', options=(('AAPL', 0), ('AMC', 1), ('AMZN', 2), ('BB', 3), ('GME', 4), ('…

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

FloatRangeSlider(value=(0.0, 100.0), continuous_update=False, description='Price Range:')

FloatRangeSlider(value=(0.0, 1000000.0), continuous_update=False, description='Volume Range:', max=1000000.0, …

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>